# Task 3: Interaction of pedestrians

Now the pedestrians have to interact with each other. **Insert five pedestrians, roughly equally spaced on a circle in a fairly large distance (30-50m) around a single target in the center**
of the scenario. Run the scenario and report your findings. What is the configuration of the pedestrians around
the target after the simulation? **It is also possible to have them removed entirely, if you made your target
absorbing?** Do the pedestrians all **reach the target roughly at the same time?** They should, because they
start at the same distance! Be careful that distance here does not mean number of cells, but Euclidean
distance. If not, implement a way to correctly traverse the space in arbitrary directions with roughly the same
speed.

In [ ]:
# always import addroot first
import addroot

In [ ]:
# we will import everything else here
from src.ca import CrowdModelCellularAutomaton
from src.notebook_utils import get_n_equally_spaced_points_on_grid_circle
from src.config import get_save_figure_function
import numpy as np

save_current_figure = get_save_figure_function("3_movement_directions")


# set matplotlib to interactive mode -> requires ipympl to be installed
%matplotlib widget

%reload_ext autoreload
%autoreload 2

### 1. Construct the scenario.

Get the required number of cells for a circle with radius of 30m:

In [ ]:
radius = 30  # meters
cell_unit = 0.4  # meters
cell_radius = int(radius / cell_unit)
cell_diameter = cell_radius * 2
cell_grid = cell_diameter + 2  # circle should fit on this many cell units across (+ 1 on each side to make sure)
print("Square grid with a side length of", cell_grid, "cells.")

We want to put a target at the center of the circle:

In [ ]:
center = int(cell_grid / 2) - 1
circle_center = [center, center]

Now want equally spaced pedestrians on the circle:
 - first we need to get coordinates that match these requirements. We will use our function `get_n_equally_spaced_points_on_grid_circle(n, center, radius)`, which will return an array containing `n` randomly rotated discrete point coordinates approximately equally spaced and approximately on the continuous circle with given `center` and `radius`.

In [ ]:
pedestrian_locations = get_n_equally_spaced_points_on_grid_circle(n=5, center=circle_center, radius=cell_radius)
display(pedestrian_locations)

 - now we can add the pedestrians at the given cells.
 - to make sure they even have a chance at reaching the target roughly at the same time, of course we must give them equal speeds.
 - Since the implementation of varying speeds relies on a stochastic process, here we simply set their speeds to the maximum speed, which essentially makes them move at each time step, making the results slightly more deterministic.

In [ ]:
def get_scenario_model(absorbing, save=True):
    ca = CrowdModelCellularAutomaton(grid_size=(cell_grid, cell_grid), absorbing_targets=absorbing)
    ca.set_cell_target(circle_center)
    # sample new pedestrian locations
    pedestrian_locations = get_n_equally_spaced_points_on_grid_circle(5, circle_center, cell_radius)
    for i in range(pedestrian_locations.shape[0]):
        ca.set_cell_pedestrian(pedestrian_locations[i, :], max_speed_fraction=1.0)
    if save:
        ca.save_state()
    return ca

### 2. Simulation (non-absorbing).

Now that we are done constructing the scenario we can save the state and simulate the movement:

In [ ]:
ca1 = get_scenario_model(absorbing=False)
ca1.plot_state(fig_size=(10, 10))
save_current_figure("scenario")

simulation_seconds = 15

In [ ]:
ca1.simulate(start_at_saved_state=True, seconds=simulation_seconds)

In [ ]:
ca1.plot_simulation_end_state(fig_size=(10, 10))
save_current_figure("non_absorbing_end_state")

In [ ]:
ca1.plot_simulation_with_time_slider(fig_size=(10, 10))

In [ ]:
# once you have selected a slider position that you like:
#save_current_figure("non_absorbing_slider")

### 3. Simulation (absorbing).

We can now construct the scenario again and set the target to be absorbing:

In [ ]:
ca2 = get_scenario_model(absorbing=True)

And run the simulation:

In [ ]:
ca2.simulate(start_at_saved_state=True, seconds=simulation_seconds)

In [ ]:
ca2.plot_simulation_end_state(fig_size=(10, 10))
save_current_figure("absorbing_end_state")

In [ ]:
ca2.plot_simulation_with_time_slider(fig_size=(10, 10))

In [ ]:
# once you have selected a slider position that you like:
#save_current_figure("absorbing_slider")

### 4. Do they reach the target roughly at the same time?

We can use the plot with the slider to check this interactively. Based on the slider plot, the pedestrians seems to arrive at the target roughly at the same time. But let us examine this more analytically.

Now that the targets are absorbing, we save the time at which the pedestrian is absorbed as its evacuation time. So we can check the difference between the evacuation times: 

In [ ]:
evacuation_times = ca2.get_simulation_evacuation_times_per_pedestrian()  # {id_1: evac_time_1, ...}
evacuation_times = sorted(evacuation_times.values())  # [evac_time_1, ...]
print("Evacuation times:", end=" ")
for t in evacuation_times:
    print(f"{t:.4}s", end=" ")
ratio_fastest_slowest = min(evacuation_times) / max(evacuation_times)
print(f"\nThe fastest pedestrian reached the target in {100 * (ratio_fastest_slowest):.3}%"
      f" of the time it took the slowest one.")

However, the movement of the pedestrians is limited to the square grid. The optimal path from the radius to the center is usually not possible, unless that path is either on a single axis, or exactly in the diagonal. Therefore, there is also a variation in the path distance travelled by the 5 pedestrians. We can check the walked distances:

In [ ]:
walked_distances = ca2.get_simulation_walked_distance_per_pedestrian()  # {id_1: dist_1, ...}
walked_distances = sorted(walked_distances.values())  # [dist_1, ...]
print("Walked distances:", end=" ")
for d in walked_distances:
    print(f"{d:.4}m", end=" ")
ratio_closest_to_furthest = min(walked_distances) / max(walked_distances)
print(f"\nThe closest pedestrian also had to walk only {100 * (ratio_closest_to_furthest):.3}%"
      f" of the distance walked by the furthest away one.")

Therefore, this unavoidable difference in path distance (property of the modeling abstraction) probably accounts for at least some of the difference in evacuation time. To check if that is the case, we can look at the average speeds of the pedestrians, since these already account for the distance walked. The average speed per pedestrian can be obtained directly (even when the target is non-absorbing):

In [ ]:
speeds = ca2.get_simulation_average_speed_per_pedestrian()  # {id_1: speed_1, ...}
speeds = list(speeds.values())  # [speed_1, ...]
print("Speeds:", end=" ")
for s in speeds:
    print(f"{s:.4}m/s", end=" ")
ratio_fastest_slowest = max(speeds) / min(speeds)
print(f"\nThe fastest pedestrian was only {100 * (ratio_fastest_slowest - 1):.3}% faster than the slowest one.")

We can clearly see that the speeds have a smaller difference than the evacuation times, confirming the hypothesis above. 

Yet, even without justification, the difference in time to reach the target is still within a reasonably small margin of error. Since the placement of the points around the circle contains some randomness, we can repeat the experiment a couple of times and report the mean and maximum difference: 

In [ ]:
min_max_time_percentages = []
print("Running...")
for _ in range(5):
    ca3 = get_scenario_model(absorbing=True, save=False)
    ca3.simulate(start_at_saved_state=False, seconds=simulation_seconds, verbose=False)
    evacuation_times = list(ca3.get_simulation_evacuation_times_per_pedestrian().values())
    min_max_time_percentages.append(min(evacuation_times) / max(evacuation_times))
print(f"On average, the fastest pedestrian took {np.mean(min_max_time_percentages) * 100:.3}%"
      f" of the time it took the slowest one.")
print(f"At worst, it took the fastest pedestrian {min(min_max_time_percentages) * 100:.3}%"
      f" of the time it took the slowest one.")

The worst difference seems to always be above 90% (all of the times I checked), which seems acceptable.

Done!